In [1]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

A new version of galpy (1.9.2) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


# Read in cluster data

In [2]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
# limit to only necessary columns
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
# rename columns if necessary to be consistent with the package
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']

#m6_reference_frame = m6[['x', 'y', 'z', 'U', 'V', 'W']].mean().values

In [3]:
# full cluster catalog
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

# Create star cluster objects

In [4]:
cr135_data = StarClusterData(cr135, data_name = 'Cr135 family', min_size = 0, max_size = 5, color = 'orange', opacity = 1)
m6_data = StarClusterData(m6, data_name = 'M6 family', min_size = 0, max_size = 5, color = 'cyan', opacity = 1)
ap_data = StarClusterData(ap, data_name = '⍺Per family', min_size = 0, max_size = 5, color = 'violet', opacity = 1)

sc = StarClusterCollection([cr135_data, m6_data, ap_data])
sc.integrate_all_orbits(time=np.arange(0., -61., -1.), reference_frame_center=None)

# Make 3D plot

In [5]:
# basic custom layout properties
width = 800
height = 800
x_width = 1000
y_width = x_width
z_width = 600
z_aspect = 1.*(z_width/x_width)

In [6]:
# define custom layout
        # plotly scene, layout, and camera
camera = dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0., y=-.8, z=1.5),
    projection = dict(type = 'perspective')
) 

scene = dict(
    camera = camera,
    aspectmode = 'manual',
    aspectratio = dict(x=1, y=1, z=z_aspect),
    xaxis=dict(
        title='X\' (pc)', 
        range = [-x_width, x_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = 'white',
        linewidth = 2.,
    ), 
    yaxis=dict(
        title='Y\' (pc)', 
        range = [-y_width, y_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = 'white',
        linewidth = 2.,
    ), 
    zaxis=dict(
        title='Z\' (pc)', 
        range = [-z_width, z_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        showspikes = False,
        mirror = True,
        nticks = 5,
        linecolor = 'white',
        linewidth = 2.,
    )
)

legend = dict(
    x = 0,
    y = 1,
    title = dict(
        text = 'Click to toggle traces on/off',
        font = dict(
            size = 18,  
            color = 'white'
        ),  
        side = 'top'
    ),
    font = dict(
        size = 14,
        family = 'Arial',
        color = 'white'
    ),
    itemsizing = 'constant',
    bgcolor = 'rgba(0,0,0,0)' # transparent legend
)

layout = dict(
    scene = scene,
    template = 'plotly_dark', 
    dragmode = 'turntable',
    legend = legend,
    width = width,
    height = height
)

## Create static traces with Plotly

In [7]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['X'].values[::ds_index],
    y=dfv['Y'].values[::ds_index],
    z=dfv['Z'].values[::ds_index],
    mode='markers',
    marker=dict(size=4.,
                #color='gray',
                color=dfv['extinction'].values[::ds_index],
                colorscale='Ice',
                cmin = 0.0007,
                cmax = 0.005,
                symbol='circle',
                opacity=.02),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0

dfv = pd.read_csv('/Users/cam/Downloads/vergely_dust_scatter_large.csv')
dfv = dfv.loc[dfv['e'].between(0, .01)]
e = dfv['e'].values
scatter_vergely = go.Scatter3d(
    x=dfv['X'].values[::3],
    y=dfv['Y'].values[::3],
    z=dfv['Z'].values[::3],
    mode='markers',
    marker=dict(size=7.5,
                color='gray',
                symbol='circle',
                opacity=.04),
    line = dict(color = 'gray', width = 0.),
    name='Vergeley Dust',
    visible = 'legendonly',
    hovertext='Vergeley Dust')
# static_traces.append(scatter_vergely)
# static_traces_times.append([0.]) # only show at t=0

# read in zucker clouds
zucker_clouds = pd.read_csv('/Users/cam/Downloads/zucker_clouds.csv')
scatter_zucker = go.Scatter3d(
    x=zucker_clouds['X'].values,
    y=zucker_clouds['Y'].values,
    z=zucker_clouds['Z'].values,
    mode='markers',
    marker=dict(size=5.,
                color='red',
                symbol='circle',
                opacity=.5),
    line = dict(color = 'red', width = 0.),
    name='Zucker Clouds',
    visible = True,
    hovertext=zucker_clouds['name'])
# static_traces.append(scatter_zucker)
# static_traces_times.append([0.]) # only show at t=0

hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.03),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0



In [8]:
plot_3d = StarClusters3DPlotter()
save_name = '/Users/cam/Downloads/cluster_orbits_figure.html'
fig3d = plot_3d.generate_3d_plot(collection = sc, show = False, save_name =save_name, figure_layout=layout, static_traces = static_traces, static_traces_times = static_traces_times, reference_frame_center = None)